In [1]:
import chromadb
client = chromadb.PersistentClient()

In [2]:
collection = client.create_collection(
    name="winterdev"
)

In [3]:
import pandas as pd
from tqdm import tqdm

data = pd.read_csv("econo_data.csv", encoding='cp949')
data.sample(5)

,Question,Answer
265,학적 상태에 따른 지원 자격은 어떻게 되나요?,"에코노베이션은 나이, 휴학 여부, 전공에 관계없이 IT에 관심이 있고 열정이 있다면..."
274,동아리 활동의 필수 활동 기간은 얼마입니까?,에코노베이션은 3학기 이상 활동이 원칙입니다.
381,여수캠퍼스도 지원 가능한가요?,네! 실제로 여수캠에서 활동하셨던 분도 계십니다 :)
528,Quokka 팀은 언제 발표하나요?,Quokka팀의 전남대학교 주차권 신청 시스템 프로젝트는 1부 (14:25 ~ 15...
409,너와 그린 기린 그림은 어떤 프로젝트인가요?,"*23#팀의 너와 그린 기린 그림은 WEB 프로젝트로, 대학생들을 위한 지역기반 펀..."


In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [5]:
ids = []
metadatas = []
embeddings = []

for row in tqdm(data.iterrows()):
    index = row[0]
    query = row[1].Question
    answer = row[1].Answer
    
    metadata = {
        "query": query,
        "answer": answer
    }
    
    embedding = model.encode(query, normalize_embeddings=True)
    
    ids.append(str(index))
    metadatas.append(metadata)
    embeddings.append(embedding)
    
chunk_size = 1024  # 한 번에 처리할 chunk 크기 설정
total_chunks = len(embeddings) // chunk_size + 1  # 전체 데이터를 chunk 단위로 나눈 횟수
embeddings = [ e.tolist() for e in tqdm(embeddings)]  

for chunk_idx in tqdm(range(total_chunks)):
    start_idx = chunk_idx * chunk_size
    end_idx = (chunk_idx + 1) * chunk_size
    
    # chunk 단위로 데이터 자르기
    chunk_embeddings = embeddings[start_idx:end_idx]
    chunk_ids = ids[start_idx:end_idx]
    chunk_metadatas = metadatas[start_idx:end_idx]
    
    # chunk를 collection에 추가
    collection.add(embeddings=chunk_embeddings, ids=chunk_ids, metadatas=chunk_metadatas)

551it [00:21, 26.22it/s]
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embedding ID: 22
Add of existing embedding ID: 23
Add of existing embedding ID: 24
Add of existing embedding ID: 25
Add of e

In [6]:
import numpy as np
import faiss

embeddings = np.array(embeddings)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [7]:
def retriever(question):
    query_embeddings=model.encode(question, normalize_embeddings=True, convert_to_tensor=True)
    
    top_k = 5
    distances, indices = index.search(np.expand_dims(query_embeddings, axis=0), top_k)
    
    temp = data.iloc[indices[0]]
    temp.loc[:, 'distances'] = distances[0]
    similar = temp[temp['distances'] < 1.1]
    
    result = {'Question' : similar['Question'].tolist(),
             'Answer' : similar['Answer'].tolist()}
    
    prompt = f"""
    에코노베이션이라는 동아리의 문의 해결을 도와주세요. You are an intelligent assistant helping the users with their questions
    Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.
    The instructions should be in Korean. Reply via text only
 
    CONTEXT: 
    {result}
 
    QUESTION:
    {question}
    
    Do not try to make up an answer:
     - If the answer to the question cannot be determined from the context alone, say "해당 질문은 https://econovation.kr/contact 혹은 에코노베이션 카카오톡 채널 https://pf.kakao.com/_laTLs로 문의주세요!"
     - If the context is empty, just say "현재 서비스 준비중으로 답변드리기 어렵습니다."
     
    Strictly Use ONLY the following pieces of context to answer the question at the end.
    Helpful Answer:
    """
    
    return prompt

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import os
import openai

os.environ["OPENAI_API_KEY"] = '{openai_key}'

In [9]:
llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-0613')

In [10]:
def response(message, history):
    message = retriever(message)
    
    history_langchain_format = []
    
    for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
    
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm(history_langchain_format)
    return gpt_response.content

In [11]:
import gradio as gr

gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="대화를 입력해주세요.", container=False, scale=7),
        title="ECONOVATION CHATBOT",
        examples=[["에코노베이션에서는 어떤 활동을 할 수 있나요?"], ["어떻게 지원할 수 있나요?"], ["현재는 몇 기수까지 있어?"]],
        retry_btn="다시보내기 ↩",
        undo_btn="지난 대화 삭제 ❌",
        clear_btn="전체 대화 삭제 💫",
).launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on public URL: https://dded1a183b612fab6b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


C:\Users\kt826\AppData\Local\Temp\ipykernel_40508\1515072520.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:, 'distances'] = distances[0]
C:\Users\kt826\AppData\Local\Temp\ipykernel_40508\1515072520.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:, 'distances'] = distances[0]
C:\Users\kt826\AppData\Local\Temp\ipykernel_40508\1515072520.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value